This workbook demonstrates the complete flow from excel created / loaded scenarios to the API, including demonstrating how you can work with scenarios in pandas along the way.

Before you begin you will need to place a config.yml in your root directory. An example.config.yml is available in /examples and will guide you through the setup. The README.md has additional details on environment management if you're setting up for the first time.

In [ ]:
# Check the environment is properly configured.
from example_helpers import setup_notebook
setup_notebook()

For the purposes of this demonstration workbook, we will use the pre-filled template 'my_input_excel.xlsx' which demonstrates a few of the possibilities afforded by the pyetm package. If it's your first time using the tool, have a look at the excel to get a sense of the structure.

In the example, there are two scenarios, with short names scen_a and scen_b. You can use short names in the parameters sheet to specify which inputs belong to which scenario. Because scen_b has no scenario_id, it is being created. It will be created with all the metadata included in the sheet, plus any of the inputs under the column with its short_name and any sortables and curves specified in the sheets named beside the sortables and custom_curves rows. The same goes for scen_a, but because it has a scenario_id (1357395) that scenario will be loaded, and then updated with anything as set in the excel.

**TODO**: Figure out how to manage the fact that for this example whatever scenario 1357395 is will be constantly updated etc by everyone who wants to try running this script on pro/beta. At the moment its just a local scenario.

In [ ]:
from pyetm.models.scenarios import Scenarios

scenarios = Scenarios.from_excel("example_input_excel.xlsx")
#scenario_a = Scenario.load(123456789) #TODO: Also load a scenario and include it in the array

Now we have the 'scenarios' in pyetm which represent actual real scenarios in the ETM, one created and one loaded.

The following blocks show how you can explore these scenarios' attributes - run some if you want to explore the data structures.

In [ ]:
# Metadata
for scenario in scenarios:
    print(f"Title: {scenario.title}")
    print(f"Area: {scenario.area_code}")
    print(f"End year: {scenario.end_year}")
    print(f"Version: {scenario.version}")
    print(f"Source: {scenario.source}")
    print(f"Metadata: {scenario.metadata}")
    print("")

In [ ]:
# Inputs
for scenario in scenarios:
    inputs = scenario.inputs.to_dataframe(columns=["user", "default", "min", "max"]).head(20)
    print(inputs)
    print("")

In [ ]:
# Sortables
for scenario in scenarios:
    sortables = scenario.sortables.to_dataframe()
    print(sortables)
    print("")

In [ ]:
# Custom Curves
for scenario in scenarios:
    curves = scenario.custom_curves.to_dataframe().head(20)
    print(curves)
    print("")

In [ ]:
# Queries
for scenario in scenarios:
    print(scenario.results())
    print("")

In [ ]:
# Warnings - did anything go wrong?
for scenario in scenarios:
    print(scenario.show_all_warnings())
    print()

We can directly modify any of the attributes using Pandas, or we can re-export the scenarios to excel and make modifications that way. When exporting to excel, more data will be available than was in the input, because the ETM results will be included by default. The 'output curves' will be stored in a separate excel workbook, separated by carrier type. By default everything is included, but you can also specify what you want.

In [ ]:
# Export the scenarios to excel
scenarios.to_excel("export.xlsx") # This will create export.xlsx and export_output_curves.xlsx.

# There are also options - decide whether to include the output_curves, specify where you want them and specify which carriers you're interested in.
scenarios.to_excel(("export_with_options.xlsx"), export_output_curves=True, output_curves_path="output_curves.xlsx", carriers=["electricity", "heat", "hydrogen"])